### AE 470 - Assignment 3
## Due in 1 week (Tuesday, Febuary 25, 2025 at 11:59 pm)

Submit your solutions to Moodle in the this Jupyter notebook.  


In [1]:
# Do a "Save Notebook As..." with your name in the filename.  For example, "Collins,Eileen_assignment_02_ae470.ipynb"

# SOLUTION

# Add your collaborators names here


In [2]:
# Configure Jupyter so figures appear in the notebook
# %matplotlib inline
import matplotlib.pyplot as plt
# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

In [3]:
try:
    from modsim import *
except ImportError:
    print("Download modsim from the course repository at https://github.com/jeffwalton/AE470")

import numpy as np

## 1. $ \vec{\mathbf{r}} $, $ \vec{\mathbf{v}} $ to COE

Write a python function to calculate orbital elements from the state vectors, $ \vec{\mathbf{r}} $ and $ \vec{\mathbf{v}} $. 

In [4]:
def state_vector_to_coe(r_vec, v_vec, mu ):
    ri = r_vec[0]
    rj = r_vec[1]
    rk = r_vec[2]
    vi = v_vec[0]
    vj = v_vec[1]
    vk = v_vec[2]
    rmag = np.sqrt(ri**2+rj**2+rk**2)
    vmag = np.sqrt(vi**2+vj**2+vk**2)
    h_vec = np.cross(r_vec, v_vec)
    h = np.sqrt(h_vec[0]**2+h_vec[1]**2+h_vec[2]**2)
    e_vec = 1/mu*(np.cross(v_vec, h_vec))-r_vec/rmag
    ecc = np.sqrt(e_vec[0]**2+e_vec[1]**2+e_vec[2]**2)
    p = h**2/mu
    radtheta = np.arccos((p-rmag)/(ecc*rmag))
    theta = np.degrees(radtheta)
    A_vec = [0, 0, 1]
    vec = np.cross(A_vec, h_vec)
    A = np.sqrt(vec[0]**2+vec[1]**2+vec[2]**2)
    if vec[1] >= 0:
        radraan = np.arccos(vec[0]/A)
    else:
        radraan = 2*np.pi-np.arccos(vec[0]/A)
    raan = np.degrees(radraan)
    radincl = np.arccos(h_vec[2]/h)
    incl = np.degrees(radincl)
    if e_vec[2] >= 0:
        radomega = np.arccos(np.dot(vec, e_vec)/(A*ecc))
    else:
        radomega = 2*np.pi-np.arccos(np.dot(vec, e_vec)/(A*ecc))
    omega = np.degrees(radomega)
    
    
    return h, ecc, theta, raan, incl, omega

## 2. COE to $ \vec{\mathbf{r}} $, $ \vec{\mathbf{v}} $

Write a python function to calculate the state vectors, $ \vec{\mathbf{r}} $ and $ \vec{\mathbf{v}} $, from orbital elements.

In [5]:
def coe_to_state_vector(h, ecc, theta, raan, incl, omega, mu ):
    radincl = np.radians(incl)
    radraan = np.radians(raan)
    radomega = np.radians(omega)
    radtheta = np.radians(theta)
    r = ((h**2/mu)/(1+ecc*np.cos(radtheta)))
    rp = Vector(r*np.cos(radtheta), r*np.sin(radtheta), 0)
    vp = Vector(-mu/h*np.sin(radtheta), mu/h*(ecc+np.cos(radtheta)), 0)
    def C1(theta):
        c = np.cos(theta)
        s = np.sin(theta)
        return np.matrix([[ 1, 0, 0],
            [0, c, s],
            [0, -s, c]])

    def C2(theta):
        c = np.cos(theta)
        s = np.sin(theta)
        return np.matrix([[ c, 0, -s],
                [0, 1, 0],
                [s, 0, c]])

    def C3(theta):
        c = np.cos(theta)
        s = np.sin(theta)
        return np.matrix([[ c, s, 0],
                [-s, c, 0],
                [0, 0, 1]])

    peri_geo = (C3(radomega) @ C1(radincl) @ C3(radraan)).transpose()

    r_vec = peri_geo @ rp
    v_vec = peri_geo @ vp


    
    return r_vec, v_vec

## 3. Test Functions

Test your functions on the following state vectors and orbital elements.

$ \vec{\mathbf{r}} = -6045 \hat{\mathbf{I}} - 3490 \hat{\mathbf{J}} + 2500 \hat{\mathbf{K}} $ km 

$ \vec{\mathbf{v}} = -3.457 \hat{\mathbf{I}} + 6.618 \hat{\mathbf{J}} + 2.533 \hat{\mathbf{K}} $ km/s

$ h = 58,310 $ km^2/s

$ i = 153.2^{\circ} $,

$ \Omega = 255.3^{\circ} $, 

$ e = 0.1712 $, 

$ \omega = 20.07^{\circ} $,

$ \theta = 28.45^{\circ} $



In [8]:
#state to coe test

r_vec = [-6045, -3490, 2500]
v_vec = [-3.457, 6.618, 2.533]
mu = 3.986*10**5
ans = state_vector_to_coe(r_vec, v_vec, mu)

(58311.66993185606,
 0.17121234628445364,
 28.44562830661485,
 255.27928533439618,
 153.2492285182475,
 20.068316650582524)

In [7]:
# coe to state test
h = 58310
incl = 153.2
raan = 255.3
ecc = 0.1712
omega = 20.07
theta = 28.45
mu = 3.986*10**5
answer = coe_to_state_vector(h, ecc, theta, raan, incl, omega, mu )

(array([-6041.70115028, -3491.89729502,  2504.38823005]),
 array([-3.45671638,  6.61670935,  2.53710431]))